In [1]:
# 셀 1: 필수 패키지 설치
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]

In [2]:
def scroll_through_all_lis(driver):
    """모든 li 요소를 순차적으로 스크롤하여 lazy loading을 유도"""
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 모든 li를 대상으로 하나씩 스크롤
        lis = driver.find_elements("css selector", "section.CommonPcListUnitProduct_list_unit_product__AeuOT ul[role='list'] > li")
        for li in lis:
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", li)
                time.sleep(0.3)  # 로딩 대기
            except:
                continue

        # 새 높이를 확인해 더 로딩되는지 판단
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


In [3]:
# 셀 2: Selenium 설정 및 데이터 수집
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# ✅ 크롤링 대상 URL 리스트
urls = [
    "https://www.lge.co.kr/category/wash-tower",
    "https://www.lge.co.kr/category/wash-combo",
    "https://www.lge.co.kr/category/washing-machines",
    "https://www.lge.co.kr/category/dryers"
]

# ✅ Chrome 옵션 설정 (Colab 환경)
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ✅ 크롬 드라이버 실행
driver = webdriver.Chrome(options=options)

# ✅ 스크래핑 함수 정의
def scrape_data_ec_product(url):
    driver.get(url)
    time.sleep(3)  # 페이지 로딩 대기

    scroll_through_all_lis(driver)  # ✅ 실제 li를 보이게 함

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    section = soup.find('section', class_="CommonPcListUnitProduct_list_unit_product__AeuOT")
    data_list = []

    if section:
        ul = section.find('ul', role='list')
        if ul:
            lis = ul.find_all('li')
            for li in lis:
                data = li.get('data-ec-product')
                if data:
                    data_list.append(data)
    return data_list

# ✅ 전체 URL 대상 스크래핑 수행
all_data = {}
for url in urls:
    print(f"크롤링 중: {url}")
    data = scrape_data_ec_product(url)
    all_data[url] = data
    print(f"수집된 항목 수: {len(data)}")

크롤링 중: https://www.lge.co.kr/category/wash-tower
수집된 항목 수: 89
크롤링 중: https://www.lge.co.kr/category/wash-combo
수집된 항목 수: 26
크롤링 중: https://www.lge.co.kr/category/washing-machines
수집된 항목 수: 266
크롤링 중: https://www.lge.co.kr/category/dryers
수집된 항목 수: 102


In [4]:
# 셀 3: 결과 확인
import json

# ✅ 결과를 JSON 형식으로 출력
for url, items in all_data.items():
    print(f"\n📌 URL: {url}")
    for item in items:
        print(json.loads(item))  # JSON 문자열을 dict로 변환하여 출력


📌 URL: https://www.lge.co.kr/category/wash-tower
{'model_name': 'LG 트롬 오브제컬렉션 워시타워', 'model_id': 'MD09942859', 'model_sku': 'WL21WDU.AKOR', 'model_gubun': '일반제품', 'base_price': '3,100,000', 'price': '2,715,600', 'discounted_price': '3,100,000', 'grand_price': '3,990,000', 'brand': 'LG', 'category': '생활가전/워시타워/워시타워', 'ct_id': 'CT50000110', 'model_super_category': '생활가전', 'model_category': '워시타워', 'model_sub_category': '워시타워', 'delivery_badge': 'N', 'price_badge': 'Y'}
{'model_name': 'LG 트롬 오브제컬렉션 워시타워', 'model_id': 'MD09942827', 'model_sku': 'WL21EGZU.AKOR', 'model_gubun': '일반제품', 'base_price': '3,260,000', 'price': '2,864,400', 'discounted_price': '3,260,000', 'grand_price': '4,290,000', 'brand': 'LG', 'category': '생활가전/워시타워/워시타워', 'ct_id': 'CT50000110', 'model_super_category': '생활가전', 'model_category': '워시타워', 'model_sub_category': '워시타워', 'delivery_badge': 'N', 'price_badge': 'Y'}
{'model_name': 'LG 트롬 오브제컬렉션 워시타워', 'model_id': 'MD09900826', 'model_sku': 'W20WHN.AKOR', 'model_gubun'

In [5]:
driver = webdriver.Chrome(options=options)

def get_product_links(url):
    driver.get(url)
    time.sleep(3)  # 페이지 로딩 대기

    scroll_through_all_lis(driver)  # ✅ 실제 li를 보이게 함

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = []
    divs = soup.find_all('div', class_="CommonPcListUnitProduct_img_wrap__4haVO")
    for div in divs:
        a_tag = div.find('a')
        if a_tag and a_tag.has_attr('href'):
            links.append(a_tag['href'])
    return links

def get_base_url(url):
    # /category가 있다면 그 앞까지만 남김
    idx = url.find('/category')
    if idx != -1:
        return url[:idx]
    else:
        # /category가 없으면 전체 url
        return url

all_product_links = {}

for url in urls:
    print(f"크롤링 중: {url}")
    product_links = get_product_links(url)
    print(f"수집된 링크 수: {len(product_links)}")

    # /category 뒤로 다 삭제한 base_url 만들기
    base_url = get_base_url(url)

    full_links = []
    for link in product_links:
        if link.startswith('/'):
            full_url = base_url + link
        else:
            full_url = base_url + '/' + link
        full_links.append(full_url)

    all_product_links[url] = full_links

# 결과 출력
print("\n수집된 전체 상품 상세페이지 URL 리스트 (카테고리별):")
for url, links in all_product_links.items():
    print(f"\n📌 카테고리 URL: {url}")
    for link in links:
        print(link)

크롤링 중: https://www.lge.co.kr/category/wash-tower
수집된 링크 수: 89
크롤링 중: https://www.lge.co.kr/category/wash-combo
수집된 링크 수: 26
크롤링 중: https://www.lge.co.kr/category/washing-machines
수집된 링크 수: 266
크롤링 중: https://www.lge.co.kr/category/dryers
수집된 링크 수: 102

수집된 전체 상품 상세페이지 URL 리스트 (카테고리별):

📌 카테고리 URL: https://www.lge.co.kr/category/wash-tower
https://www.lge.co.kr/wash-tower/wl21wdu
https://www.lge.co.kr/wash-tower/wl21egzu
https://www.lge.co.kr/wash-tower/w20whn
https://www.lge.co.kr/wash-tower/w20wznm
https://www.lge.co.kr/wash-tower/w20egz
https://www.lge.co.kr/wash-tower/wl22kdu
https://www.lge.co.kr/wash-tower/wl22wdu
https://www.lge.co.kr/wash-tower/wl22ymzu
https://www.lge.co.kr/wash-tower/wl22gmzu
https://www.lge.co.kr/wash-tower/wl22eghu
https://www.lge.co.kr/wash-tower/wl22myzu
https://www.lge.co.kr/wash-tower/wl22emzu
https://www.lge.co.kr/wash-tower/w10bn
https://www.lge.co.kr/wash-tower/w10bhn
https://www.lge.co.kr/wash-tower/w10wan
https://www.lge.co.kr/wash-tower/w20wanq
htt

In [6]:
# def extract_desktop_gallery_imgs(soup):
#     """
#     1. div.pdp-visual-area.hidden-sm#desktop_summary_gallery > div.pdp-visual-inner.default 아래의 모든 img src
#     2. div.pdp-thumbnail-nav 아래의 모든 img src
#     모두 리스트로 반환
#     """
#     main_imgs = []
#     thumb_imgs = []

#     # 1. 메인 갤러리 이미지
#     main_area = soup.find('div', class_='pdp-visual-area hidden-sm', id='desktop_summary_gallery')
#     if main_area:
#         inner = main_area.find('div', class_='pdp-visual-inner default')
#         if inner:
#             for img in inner.find_all('img'):
#                 if img.has_attr('src'):
#                     main_imgs.append(img['src'])
#                 elif img.has_attr('data-src'):
#                     main_imgs.append(img['data-src'])

#     # 2. 썸네일 이미지
#     thumb_area = soup.find('div', class_='pdp-thumbnail-nav')
#     if thumb_area:
#         for img in thumb_area.find_all('img'):
#             if img.has_attr('src'):
#                 thumb_imgs.append(img['src'])
#             elif img.has_attr('data-src'):
#                 thumb_imgs.append(img['data-src'])

#     return main_imgs, thumb_imgs


# for url, links in all_product_links.items():
#     print(f"\n📌 카테고리 URL: {url}")
#     for link in links:
#         print(f"  상세페이지: {link}")
#         driver.get(link)
#         time.sleep(2)
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         main_imgs, thumb_imgs = extract_desktop_gallery_imgs(soup)
#         print("메인 갤러리 이미지 src:", main_imgs)
#         print("썸네일 이미지 src:", thumb_imgs)



📌 카테고리 URL: https://www.lge.co.kr/category/wash-tower
  상세페이지: https://www.lge.co.kr/wash-tower/wl21wdu
메인 갤러리 이미지 src: ['/kr/images/wash-tower/md09942859/gallery/medium-interior01.jpg']
썸네일 이미지 src: ['/kr/images/wash-tower/md09942859/gallery/small-interior01.jpg', '/kr/images/wash-tower/md09942859/gallery/small01.jpg', '/kr/images/wash-tower/md09942859/gallery/small02.jpg', '/kr/images/wash-tower/md09942859/gallery/small03.jpg', '/kr/images/wash-tower/md09942859/gallery/small04.jpg', '/kr/images/wash-tower/md09942859/gallery/small05.jpg']
  상세페이지: https://www.lge.co.kr/wash-tower/wl21egzu


KeyboardInterrupt: 

In [7]:
driver = webdriver.Chrome(options=options)

BASE_URL = "https://www.lge.co.kr"

def get_soup(driver, url, scroll=False):
    """
    Selenium 드라이버로 url 접속 후 BeautifulSoup 객체 반환
    """
    driver.get(url)
    time.sleep(2)
    # 필요시 스크롤 등 추가 동작
    return BeautifulSoup(driver.page_source, 'html.parser')

def make_absolute_url(src, base_url=BASE_URL):
    """
    이미지 src가 절대경로가 아니면 base_url을 붙여 절대경로로 변환
    """
    if src.startswith("http"):
        return src
    return base_url + src

def extract_images(soup):
    """
    상세페이지에서 대표 이미지 src 추출 (절대경로로 반환)
    """
    images = []
    div = soup.find('div', class_="slide-content pdp-visual ui_carousel_list ui_static draggable")
    if div:
        ul = div.find('ul', class_="slide-track pdp-visual-list ui_carousel_track ui_static")
        if ul:
            for li in ul.find_all('li'):
                li_class = ' '.join(li.get('class', []))
                if (li_class == "slide-conts ui_carousel_slide default thumbnail img_square" or
                    li_class == "slide-conts ui_carousel_slide default thumbnail img_square ui_carousel_current on"):
                    a_tag = li.find('a')
                    if a_tag:
                        img_tag = a_tag.find('img')
                        if img_tag and img_tag.has_attr('src'):
                            img_src = make_absolute_url(img_tag['src'])
                            images.append({'img_src': img_src})
    return images

def extract_specs(soup):
    """
    상세페이지에서 스펙 정보 추출
    """
    specs = []
    prod_spec_div = soup.find('div', class_='prod-spec-detail')
    if prod_spec_div:
        box_div = prod_spec_div.find('div', class_='box')
        spec_title = box_div.find('h3', class_='tit').get_text(strip=True) if box_div and box_div.find('h3', class_='tit') else ''
        spec_info_div = prod_spec_div.find('div', class_='spec-info-list')
        if spec_info_div:
            ul_tag = spec_info_div.find('ul')
            if ul_tag:
                for li in ul_tag.find_all('li'):
                    dl_tag = li.find('dl')
                    if dl_tag:
                        dt_tag = dl_tag.find('dt')
                        dd_tag = dl_tag.find('dd')
                        dt_text = ''
                        dt_btn_data = None
                        if dt_tag:
                            button_tag = dt_tag.find('button')
                            if button_tag and button_tag.has_attr('data-spec-description'):
                                dt_btn_data = button_tag['data-spec-description']
                            dt_text = dt_tag.get_text(strip=True)
                        dd_text = dd_tag.get_text(strip=True) if dd_tag else ''
                        specs.append({
                            'spec_title': spec_title,
                            'dt': dt_text,
                            'dd': dd_text,
                            'dt_button_data_spec_description': dt_btn_data
                        })
    return specs

def get_product_detail_info(driver, detail_url):
    """
    상세페이지에서 이미지와 스펙 정보를 모두 추출
    """
    soup = get_soup(driver, detail_url, scroll=False)
    return {
        'images': extract_images(soup),
        'specs': extract_specs(soup)
    }

# --- 크롤링 실행 예시 ---

all_product_details = {}

for category_url, detail_urls in all_product_links.items():
    print(f"\n📂 카테고리: {category_url}")
    details_list = []
    for detail_url in detail_urls:
        info = get_product_detail_info(driver, detail_url)
        details_list.append({
            'detail_url': detail_url,
            'images': info['images'],
            'specs': info['specs']
        })
    all_product_details[category_url] = details_list

# 결과 출력
for category_url, details_list in all_product_details.items():
    print(f"\n=== 카테고리: {category_url}")
    for detail in details_list:
        print(f"\n상품 상세 URL: {detail['detail_url']}")
        print("이미지 정보:")
        for img in detail['images']:
            print(f"  - src: {img['img_src']}")
        print("스펙 정보:")
        for spec in detail['specs']:
            print(f"  - [{spec['dt']}] {spec['dd']} (button data: {spec['dt_button_data_spec_description']})")

# driver.quit()


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
· 트롬 : 21~23kg
· 통돌이 세탁기: 15~16kg

겉옷이나 속옷을 나눠 세탁하거나 땀 흘린 운동복, 반려동물 장난감 등 별도 세탁물이 많다면 기존 트롬에 미니워시를 추가해도 좋습니다.

3인 이상
· 트롬 : 21~25kg
· 통돌이 세탁기: 20~24kg
· 트롬 트윈워시: 28~29kg

드럼 방식은 세탁시 낙차를 이용한 여유 공간이 필요해 통돌이 세탁기보다 세탁 용량을 20~30% 크게 잡으면 좋습니다.)

상품 상세 URL: https://www.lge.co.kr/washing-machines/f15w-s84mw1
이미지 정보:
  - src: https://www.lge.co.kr/kr/images/washing-machines/md10555863/gallery/medium01.jpg
스펙 정보:
  - [세탁 용량 (kg)] 15 (button data: 1인 기준 일주일치 빨랫감은 평균 4~5kg* 으로 구성원 수와 사용패턴을 고려하여 용량 선택에 참고 하실 수 있습니다.

[가구 유형별 추천 용량]

1-2인
· 트롬 : 12~15kg
· 통돌이 세탁기: 10~12kg

1인 가구 일주일 치 세탁물이 4kg임을 감안해 10kg 초반 작은 용량을 추천드립니다.

2-3인
· 트롬 : 21~23kg
· 통돌이 세탁기: 15~16kg

겉옷이나 속옷을 나눠 세탁하거나 땀 흘린 운동복, 반려동물 장난감 등 별도 세탁물이 많다면 기존 트롬에 미니워시를 추가해도 좋습니다.

3인 이상
· 트롬 : 21~25kg
· 통돌이 세탁기: 20~24kg
· 트롬 트윈워시: 28~29kg

드럼 방식은 세탁시 낙차를 이용한 여유 공간이 필요해 통돌이 세탁기보다 세탁 용량을 20~30% 크게 잡으면 좋습니다.)

상품 상세 URL: https://www.lge.co.kr/washing-machines/kg24k-18w-akor2
이미지 정보:
  - src: https://www.lge.co.